In [1]:
from da_od.config import test_img
from da_od.model import DepthAnythingEstimator, MiDaSEstimator, MonocularDepthEstimator

image_path = test_img / "img-00019.jpg"

In [2]:
DepthAnything_estimator = DepthAnythingEstimator(image_path, encoder="vits")
DepthAnything_colored, DepthAnything_raw = DepthAnything_estimator.process_image()

INFO:dinov2:using MLP layer as FFN


In [3]:
MiDaS_estimator = MiDaSEstimator(image_path, model_type="DPT_Large")
MiDaS_colored, MiDaS_raw = MiDaS_estimator.process_image()

Using cache found in /Users/maryam/.cache/torch/hub/intel-isl_MiDaS_master


In [4]:
Monocular_estimator = MonocularDepthEstimator(image_path, model_name="mono_640x192")
Monocular_colored, Monocular_raw = Monocular_estimator.process_image()

/Users/maryam/Library/Caches/pypoetry/virtualenvs/depth-aware-object-detection-zcMtcJV--py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/Users/maryam/Library/Caches/pypoetry/virtualenvs/depth-aware-object-detection-zcMtcJV--py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
